**Note:** Only modfied architecture in toy model to Bidirectional RNN + Attention, added normalization and tokenization of indian language

### **Imports:**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import json
import numpy as np
import re
import string
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chenn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\chenn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# **English-Hindi:**

### **Loading Data From Drive:**

In [6]:
TRAIN_PATH = './train_data1.json'
TEST_PATH = './test_data1_final.json'

In [7]:
with open(TRAIN_PATH, 'r', encoding='utf-8') as file: # Replace this path with the dataset path in your local machine
    data = json.load(file)

source_sentences_train = []
target_sentences_train = []

source_sentences_val = []
target_sentences_val = []

id_train = []
id_val = []

for language_pair, language_data in data.items():
    if(language_pair == "English-Hindi"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              target = entry_data["target"]
              if (data_type == "Validation"):
                source_sentences_val.append(source)
                target_sentences_val.append(target)
                id_val.append(entry_id)
              else:
                source_sentences_train.append(source)
                target_sentences_train.append(target)
                id_train.append(entry_id)

with open(TEST_PATH, 'r', encoding='utf-8') as file: # Replace this path with the dataset path in your local machine
    data = json.load(file)

for language_pair, language_data in data.items():
    if(language_pair == "English-Hindi"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              if (data_type == "Test"):
                source_sentences_val.append(source)
                id_val.append(entry_id)


print(len(source_sentences_train))
print(len(target_sentences_train))

print(len(source_sentences_val))
print(len(target_sentences_val))

Language Pair: English-Hindi
  Data Type: Train
Language Pair: English-Hindi
  Data Type: Test
80797
80797
23085
0


In [8]:
x={'English':source_sentences_train,'Bengali':target_sentences_train}
df=pd.DataFrame(x)
df

English  \
0                       cancel everything on my calendar   
1      Adrenal hormone levels are at their peak durin...   
2      Golden threads are obtained from Surat, the qu...   
3              Look for agglutination within 30 seconds.   
4      The non-pompousness and informality of their l...   
...                                                  ...   
80792  So, is it that this is the optimization proble...   
80793  In this Masjid made with red stones there are ...   
80794  He began to work on the movie on August 17, 20...   
80795                          start a new shopping list   
80796                 turn off the lights in the kitchen   

                                                 Bengali  
0                       मेरे कैलेंडर पर सब कुछ रद्द करें  
1      अधिवृक्क के हार्मोन का स्तर प्रातःकाल में अपने...  
2      स्वर्ण धागे सूरत से प्राप्त होते हैं, जिनकी गु...  
3                  30 सेकेण्ड के भीतर एग्लूटिनेशन देखें।  
4      उनके जीवन की आडंबरहीनता एवं अनौपचारिकता उनके स...  
...                                                  ...  
80792   तो, यह अनुकूलन समस्या है जिसमें हम रुचि रखते थे।  
80793  लाल पत्थरों से बनायी गयी इस मस्जिद में हिन्दू ...  
80794  उन्होंने 17 अगस्त, 2010 को फिल्म पर काम करना श...  
80795                       एक नई खरीदारी सूची शुरू करें  
80796                        रसोईघर की बत्तियाँ बंद करें  

[80797 rows x 2 columns]

### **Preprocess data:**

In [10]:
pip install indic-nlp-library

Note: you may need to restart the kernel to use updated packages.


In [11]:
import string
import re
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize
import nltk

def preprocess_and_remove_punctuation(sentence):
    sentence = ''.join([char for char in sentence if char not in string.punctuation and not char.isdigit()])
    return sentence

def preprocess_english(sentences):
    tokenized_sentences = [nltk.word_tokenize(preprocess_and_remove_punctuation(sentence.lower())) for sentence in sentences]
    return tokenized_sentences

def preprocess_hindi(sentences):
    normalizer = IndicNormalizerFactory().get_normalizer("hi")
    processed_sentences = []
    for sentence in sentences:
        normalized_sentence = normalizer.normalize(sentence)
        tokens = list(indic_tokenize.trivial_tokenize(normalized_sentence))
        processed_sentences.append(tokens)
    return processed_sentences

target_sentences_train = [re.sub(r'[a-zA-Z]', '', hi) for hi in target_sentences_train]

english_tokens = preprocess_english(source_sentences_train)
english_test = preprocess_english(source_sentences_val)
hindi_tokens = preprocess_hindi(target_sentences_train)
hindi_test = preprocess_hindi(target_sentences_val)

en_train = english_tokens
en_test = english_test
de_train = hindi_tokens
de_test = hindi_test


### **Vocabulary Creation**

In [13]:
import torch

# Initialize vocabularies with special tokens
en_index2word = ["<PAD>", "<SOS>", "<EOS>"]  # Vocabulary English
de_index2word = ["<PAD>", "<SOS>", "<EOS>"]

en_vocab = set(en_index2word)
de_vocab = set(de_index2word)

# Populate English vocabulary
for ds in [en_train, en_test]:
    for sent in ds:
        en_vocab.update(sent)  # Add all tokens from the sentence to the set

# Populate Hindi vocabulary
for ds in [de_train, de_test]:
    for sent in ds:
        de_vocab.update(sent)

en_index2word = list(en_vocab)
de_index2word = list(de_vocab)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
en_word2index = {token: idx for idx, token in enumerate(en_index2word)}
de_word2index = {token: idx for idx, token in enumerate(de_index2word)}


In [14]:
len(en_word2index)

64778

In [15]:
en_lengths = sum([len(sent) for sent in en_train])/len(en_train)
de_lengths = sum([len(sent) for sent in de_train])/len(de_train)
print(en_lengths)
print(de_lengths)

16.307808458234835
20.531764793247273


### **Encoding sentences into integer indices, adding padding or truncating as needed.**


In [17]:
def encode_and_pad(vocab, sent, max_length):
    sos = [vocab["<SOS>"]]
    eos = [vocab["<EOS>"]]
    pad = [vocab["<PAD>"]]

    if len(sent) < max_length - 2: # -2 for SOS and EOS
        n_pads = max_length - 2 - len(sent)
        encoded = [vocab[w] for w in sent]
        return sos + encoded + eos + pad * n_pads
    else: # sent is longer than max_length; truncating
        encoded = [vocab[w] for w in sent]
        truncated = encoded[:max_length - 2]
        return sos + truncated + eos

seq_length = 20

en_train_encoded = [encode_and_pad(en_word2index, sent, seq_length) for sent in en_train]
en_test_encoded = [encode_and_pad(en_word2index, sent, seq_length) for sent in en_test]
de_train_encoded = [encode_and_pad(de_word2index, sent, seq_length) for sent in de_train]
de_test_encoded = [encode_and_pad(de_word2index, sent, seq_length) for sent in de_test]

In [18]:
en_train_encoded[1]

[6517,
 62329,
 27569,
 18989,
 18935,
 43681,
 43305,
 1660,
 512,
 57044,
 23065,
 40624,
 20753,
 64746,
 512,
 57044,
 28745,
 34924,
 31128,
 52905]

In [19]:
batch_size = 50

train_x = np.array(en_train_encoded)
train_y = np.array(de_train_encoded)
test_x = np.array(en_test_encoded)
test_y = np.array(de_test_encoded)

train_ds = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_ds = TensorDataset(torch.from_numpy(test_x))


train_dl = DataLoader(train_ds, shuffle=True, batch_size=batch_size, drop_last=True)
#test_dl = DataLoader(test_ds, shuffle=True, batch_size=batch_size, drop_last=True)

In [20]:
train_x[1]

array([ 6517, 62329, 27569, 18989, 18935, 43681, 43305,  1660,   512,
       57044, 23065, 40624, 20753, 64746,   512, 57044, 28745, 34924,
       31128, 52905])

In [21]:
train_ds[1]

(tensor([ 6517, 62329, 27569, 18989, 18935, 43681, 43305,  1660,   512, 57044,
         23065, 40624, 20753, 64746,   512, 57044, 28745, 34924, 31128, 52905],
        dtype=torch.int32),
 tensor([ 7040, 19996,  7075, 51594, 37671,  6966, 33767, 58834, 19556, 45293,
         58689, 27961, 21213, 63007, 47416, 18451,  6073, 26897, 41693, 56722],
        dtype=torch.int32))

### **Model:**

1. GRU-based encoder transforms input sentences to hidden states.



    class EncoderRNN(nn.Module):
     def __init__(self, input_size, hidden_size):



2. GRU-based decoder generates output sequences from hidden states.


    class DecoderRNN(nn.Module):
      def __init__(self, hidden_size, output_size):
     



In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#Allows the decoder to "focus" on specific parts of the input sequence while generating each word of the output sequence. This overcomes the limitations of compressing the entire input sequence into a single fixed-length vector.
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 3, hidden_size)  # hidden_size * 3 for concatenation
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.size(0)
        seq_len = encoder_outputs.size(1)

        # Expand hidden state to match encoder outputs
        hidden = hidden.transpose(0, 1)  # (batch_size, 1, hidden_size)
        hidden_exp = hidden.expand(batch_size, seq_len, -1)  # (batch_size, seq_len, hidden_size)

        # Concatenate hidden and encoder outputs for scoring
        energy = torch.tanh(self.attn(torch.cat((hidden_exp, encoder_outputs), dim=2)))  # (batch_size, seq_len, hidden_size)
        v = self.v.repeat(batch_size, 1).unsqueeze(1)  # (batch_size, 1, hidden_size)
        scores = torch.bmm(v, energy.transpose(1, 2)).squeeze(1)  # (batch_size, seq_len)

        # Compute attention weights
        attn_weights = F.softmax(scores, dim=1)  # (batch_size, seq_len)
        return attn_weights


#Encodes the input sequence into a hidden representation.
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.2):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size, padding_idx=0)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden):
        embedded = self.embedding(input)  # (batch_size, seq_len, hidden_size)
        output, hidden = self.gru(embedded, hidden)  # (batch_size, seq_len, hidden_size*2)
        output = self.dropout(output)
        # Combine bidirectional hidden states
        hidden = hidden.view(2, hidden.size(1), -1).sum(dim=0, keepdim=True)  # (1, batch_size, hidden_size)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(2, batch_size, self.hidden_size).to(device)  # 2 for bidirectional


# Decodes this representation to generate the target sequence.
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size, padding_idx=0)
        self.attention = Attention(hidden_size)
        self.gru = nn.GRU(hidden_size * 3, hidden_size)  # hidden_size * 3 accounts for context and embedded input
        self.fc = nn.Linear(hidden_size + hidden_size * 2, hidden_size)  # context + hidden state
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).transpose(0, 1)  # (batch_size, 1, hidden_size)
        embedded = F.relu(self.dropout(embedded))

        # Compute attention weights and context vector
        attn_weights = self.attention(hidden, encoder_outputs)  # (batch_size, seq_len)
        attn_weights = attn_weights.unsqueeze(1)  # (batch_size, 1, seq_len)
        context = torch.bmm(attn_weights, encoder_outputs)  # (batch_size, 1, hidden_size*2)

        # Concatenate embedded input and context vector
        rnn_input = torch.cat((embedded, context), dim=2)  # (batch_size, 1, hidden_size*3)
        output, hidden = self.gru(rnn_input.transpose(0, 1), hidden)  # (1, batch_size, hidden_size)

        # Combine output and context for prediction
        output = torch.cat((output[0], context.squeeze(1)), dim=1)  # (batch_size, hidden_size + hidden_size*2)
        output = F.relu(self.fc(output))  # (batch_size, hidden_size)
        output = F.log_softmax(self.out(output), dim=1)  # (batch_size, output_size)
        return output, hidden

def generate_predictions():
    SOS = en_word2index["<SOS>"]
    EOS = en_word2index["<EOS>"]
    target_length = seq_length
    val_ids = [i for i, _ in data["English-Hindi"]["Test"].items()]
    val_outs = []
    for i in tqdm(range(len(test_ds))):
        encoder_hidden = encoder.initHidden(batch_size=1)
        input_tensor = test_ds[i][0].unsqueeze(dim=0).to(device)
        result = []
        with torch.set_grad_enabled(False):
            encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)
            decoder_input = torch.tensor([SOS]).unsqueeze(dim=0).to(device)
            decoder_hidden = encoder_hidden
            for di in range(1, target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)
                best = decoder_output.argmax(1)
                result.append(de_index2word[best.to('cpu').item()])
                if best.item() == EOS:
                    break
                decoder_input = best.unsqueeze(dim=0)
    
        result = [i for i in result if i not in ['<EOS>', '<PAD>', '<SOS>']]
        val_outs.append(" ".join(result))
    
    df0 = pd.DataFrame()
    df0["ID"] = val_ids
    df0["Translation"] = val_outs
    df0.to_csv('answersH.csv', index=False)
    
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define models as per your code
# (EncoderRNN, DecoderRNN, and Attention classes remain unchanged)

# Hyperparameters
hidden_size = 256
dropout_p = 0.5
learning_rate = 1e-3
epochs = 25

# Initialize models
encoder = EncoderRNN(len(en_index2word), hidden_size, dropout_p).to(device)
decoder = DecoderRNN(hidden_size, len(de_index2word), dropout_p).to(device)

# Optimizers
enc_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
dec_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)

# Loss function
criterion = nn.CrossEntropyLoss()

# Mixed precision scaler
scaler = GradScaler()

# Track best model
best_loss = float('inf')
best_encoder_path = "best_encoder_hindi.pth"
best_decoder_path = "best_decoder_hindi.pth"

# Training loop with tqdm
for epoch in range(epochs):
    encoder.train()
    decoder.train()
    epoch_loss = 0
    print(f"Epoch {epoch + 1}/{epochs}")

    for idx, batch in enumerate(tqdm(train_dl, desc="Training Batches")):
        input_tensor = batch[0].to(device, non_blocking=True)  # Pin memory optimization
        target_tensor = batch[1].to(device, non_blocking=True)
        batch_size = input_tensor.size(0)

        encoder_hidden = encoder.initHidden(batch_size)

        # Zero gradients
        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()

        # Enable mixed precision
        with autocast():
            # Encode input sequence
            encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)

            # Initialize decoder input and hidden state
            decoder_input = target_tensor[:, 0].unsqueeze(0)  # SOS token
            decoder_hidden = encoder_hidden
            dec_result = torch.zeros(batch_size, target_tensor.size(1), len(de_index2word)).to(device)

            # Decode sequence
            for t in range(1, target_tensor.size(1)):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
                dec_result[:, t, :] = decoder_output
                decoder_input = target_tensor[:, t].unsqueeze(0)  # Teacher forcing

            # Compute loss
            scores = dec_result[:, 1:, :].reshape(-1, dec_result.shape[2]).float()  # Convert to float
            targets = target_tensor[:, 1:].reshape(-1).long()  # Convert to long
            loss = criterion(scores, targets)


        # Backpropagation with mixed precision
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=1)
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), max_norm=1)

        # Step optimizers with scaler
        scaler.step(enc_optimizer)
        scaler.step(dec_optimizer)
        scaler.update()

        # Accumulate loss
        epoch_loss += loss.item()

    # Calculate average epoch loss
    avg_epoch_loss = epoch_loss / len(train_dl)
    print(f"Epoch {epoch + 1}/{epochs}, Average Loss = {avg_epoch_loss:.4f}")

    # Save best model
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        torch.save(encoder.state_dict(), best_encoder_path)
        torch.save(decoder.state_dict(), best_decoder_path)
        print(f"Best model saved with loss {best_loss:.4f}")
        if epoch%2==0: generate_predictions()

# Load best model if needed
# encoder.load_state_dict(torch.load(best_encoder_path))
# decoder.load_state_dict(torch.load(best_decoder_path))


C:\Users\chenn\AppData\Local\Temp\ipykernel_4852\1838687655.py:146: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 1/25


Training Batches:   0%|                                                                       | 0/1615 [00:00<?, ?it/s]C:\Users\chenn\AppData\Local\Temp\ipykernel_4852\1838687655.py:172: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training Batches: 100%|████████████████████████████████████████████████████████████| 1615/1615 [35:25<00:00,  1.32s/it]


Epoch 1/25, Average Loss = 5.2583
Best model saved with loss 5.2583


100%|████████████████████████████████████████████████████████████████████████████| 23085/23085 [49:15<00:00,  7.81it/s]


...

Training Batches:   0%|                                                                       | 0/1615 [00:00<?, ?it/s]C:\Users\chenn\AppData\Local\Temp\ipykernel_4852\1838687655.py:172: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training Batches: 100%|████████████████████████████████████████████████████████████| 1615/1615 [19:46<00:00,  1.36it/s]


Epoch 24/25, Average Loss = 2.8726
Best model saved with loss 2.8726
Epoch 25/25


Training Batches: 100%|████████████████████████████████████████████████████████████| 1615/1615 [18:09<00:00,  1.48it/s]


Epoch 25/25, Average Loss = 2.8334
Best model saved with loss 2.8334


100%|████████████████████████████████████████████████████████████████████████████| 23085/23085 [14:39<00:00, 26.24it/s]


In [25]:
import torch

# Function to print GPU memory usage
def print_gpu_memory():
    allocated = torch.cuda.memory_allocated() / (1024 ** 2)  # Convert bytes to MB
    reserved = torch.cuda.memory_reserved() / (1024 ** 2)    # Convert bytes to MB
    print(f"Allocated GPU memory: {allocated:.2f} MB")
    print(f"Reserved GPU memory: {reserved:.2f} MB")

# Example usage
print_gpu_memory()


Allocated GPU memory: 1376.72 MB
Reserved GPU memory: 3002.00 MB


In [26]:
# Load saved model weights
encoder.load_state_dict(torch.load(best_encoder_path))
decoder.load_state_dict(torch.load(best_decoder_path))

# Ensure the models are in training mode
encoder.train()
decoder.train()

# Continue training loop
for epoch in range(10):
    epoch_loss = 0  # Track loss for the entire epoch
    for idx, batch in enumerate(train_dl):
        input_tensor = batch[0].to(device)  # Input sequence
        target_tensor = batch[1].to(device)  # Target sequence
        batch_size = input_tensor.size(0)

        encoder_hidden = encoder.initHidden(batch_size)

        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            # Encode input sequence
            encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)

            # Initialize decoder input and hidden state
            decoder_input = target_tensor[:, 0].unsqueeze(0)  # SOS token
            decoder_hidden = encoder_hidden
            dec_result = torch.zeros(batch_size, target_tensor.size(1), len(de_index2word)).to(device)

            # Decode sequence
            for t in range(1, target_tensor.size(1)):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)
                dec_result[:, t, :] = decoder_output
                decoder_input = target_tensor[:, t].unsqueeze(0)  # Teacher forcing

            # Compute loss
            scores = dec_result[:, 1:, :].reshape(-1, dec_result.shape[2])  # Exclude SOS token
            targets = target_tensor[:, 1:].reshape(-1)  # Exclude SOS token
            loss = criterion(scores, targets)

            # Backpropagation
            loss.backward()
            torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=1)
            torch.nn.utils.clip_grad_norm_(decoder.parameters(), max_norm=1)
            enc_optimizer.step()
            dec_optimizer.step()

            # Accumulate epoch loss
            epoch_loss += loss.item()

            if idx % 100 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Batch {idx}, Loss = {loss.item():.4f}")

    # Calculate average epoch loss
    avg_epoch_loss = epoch_loss / len(train_dl)
    print(f"Epoch {epoch+1}/{epochs}, Average Loss = {avg_epoch_loss:.4f}")

    # Check if current epoch has the best loss
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        torch.save(encoder.state_dict(), best_encoder_path)
        torch.save(decoder.state_dict(), best_decoder_path)
        print(f"Best model saved with loss {best_loss:.4f}")


C:\Users\chenn\AppData\Local\Temp\ipykernel_4852\3213130427.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load(best_encoder_path))
C:\Use

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Int'

In [ ]:
encoder

In [ ]:
decoder

### **Translating Validation dataset:**

In [ ]:
from tqdm import tqdm
import pandas as pd

# Load best model later (if needed)
encoder.load_state_dict(torch.load('/content/best_encoder3.5019bn.pth'))
decoder.load_state_dict(torch.load('/content/best_decoder3.5019bn.pth'))

hidden_size = 256
SOS = en_word2index["<SOS>"]
EOS = en_word2index["<EOS>"]
target_length = seq_length

val_ids = [i for i, _ in data["English-Hindi"]["Validation"].items()]
val_outs = []

for i in tqdm(range(len(test_ds))):
    encoder_hidden = encoder.initHidden(batch_size=1)
    input_tensor = test_ds[i][0].unsqueeze(dim=0).to(device)
    result = []
    with torch.set_grad_enabled(False):
        encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden)
        decoder_input = torch.tensor([SOS]).unsqueeze(dim=0).to(device)
        decoder_hidden = encoder_hidden
        for di in range(1, target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)
            best = decoder_output.argmax(1)
            result.append(de_index2word[best.to('cpu').item()])
            if best.item() == EOS:
                break
            decoder_input = best.unsqueeze(dim=0)

    result = [i for i in result if i not in ['<EOS>', '<PAD>', '<SOS>']]
    val_outs.append(" ".join(result))

df0 = pd.DataFrame()
df0["ID"] = val_ids
df0["Translation"] = val_outs
df0.to_csv('answersB.csv', index=False)
x = pd.read_csv("/content/answersB.csv")
x

In [ ]:
import pandas as pd
df1 = pd.read_csv("/content/answersB.csv") # Bengali
df2= pd.read_csv("/content/answersH.csv")  # Hindi
df3 = pd.concat([df1, df2]) #Concat

df3.to_csv('answersBH.csv', index = False)
filtered_data = pd.read_csv("/content/answersBH.csv")
answer = "/content/answer.csv"
with open(answer, "w") as f:
  f.writelines("ID\tTranslation\n")
  for i in range(filtered_data.shape[0]):
    f.writelines(f'{filtered_data["ID"][i]}\t"{filtered_data["Translation"][i]}"\n')